In [1]:
import pandas as pd

#read the two dataframes
covid_df = pd.read_csv('./files/OxCGRT_summary20200520.csv')
continent_name_df = pd.read_csv('./files/country-and-continent.csv')

### Merging the two dataframes and replacing missing values

In [2]:
#merge the dataframes on the countryName
merged_df = covid_df.merge(continent_name_df, on='CountryCode', how='left')

#Replace missing values in 'Continent_Name' column with relevant value
merged_df['Continent_Name'].fillna('Unknown', inplace=True)

### Data Cleaning and Preparation

In [3]:
#check for the countries without a continent
missing_country_continents = merged_df[merged_df['Continent_Name'] == 'Unknown']

#adds the europe continent to countryname kosovo
merged_df.loc[(merged_df['CountryName'] == 'Kosovo') & (merged_df['Continent_Name'] == 'Unknown'), 'Continent_Name'] = 'Europe'

Here I found kosovo to be the country with a valid continent so I handled it and replacing the 'unknown' variable I had inputed earlier on with the appropriate value which is europe

### Data Cleaning and Preparation Cont

In [4]:
#replace missing values in confirmedCases and confirmedDeaths columns with 0
merged_df['ConfirmedCases'].fillna(0, inplace=True)
merged_df['ConfirmedDeaths'].fillna(0, inplace=True)

#rename the columns names
merged_df.rename(columns={'Stay at home requirements': 'StayAtHome', 'School closing': 'SchoolClosing'}, inplace=True)

### A world map with bubbles to showing the total number of confirmed cases in each country

In [5]:
import plotly.express as px
import plotly.graph_objects as go

# format the date column to a datetime format for proper analysis
merged_df['Date'] = pd.to_datetime(merged_df['Date'], format='%Y%m%d')

# get the desired time range and assign it to a variable
start_date = '2020-03-01'
end_date = '2020-05-20'

# filtered dataframe of the two dates
filtered_df = merged_df[(merged_df['Date'] >= start_date) & (merged_df['Date'] <= end_date)]
filtered_df['Date'] = filtered_df['Date'].dt.strftime('%Y-%m-%d')
filtered_df = filtered_df.sort_values('Date')

fig = px.scatter_geo(filtered_df, locations='CountryCode', locationmode='ISO-3', color='Continent_Name',
                     hover_name='CountryName', size='ConfirmedCases', size_max=40,
                     projection='natural earth', animation_frame='Date',
                     animation_group='ConfirmedCases')
fig.update_geos(
   resolution=50,
    showocean=True, oceancolor="#caf0f8",
    showcountries=True, countrycolor="RebeccaPurple",
    showland=True, landcolor="#fff",
)

fig.update_layout(
    title={
        'text': 'Total Confirmed Cases by Country (1st March - 20th May)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'family': 'IBM Plex Sans'},
        'font_color': '#000'
    }
)
fig.write_html('q3.html', auto_open=True)

### Covid-19 Dashbord 

**Please note the code here answers questions 4 and 5**


In [6]:
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

date_filter = merged_df['Date'] == '2020-05-20'
filtered_data = merged_df[date_filter]

# Remove duplicate entries which was united states twice!
filtered_data = filtered_data.drop_duplicates(subset=['CountryName'])

# Sorted the data by confirmed cases in descending order
sorted_data = filtered_data.sort_values('ConfirmedCases', ascending=False)

top_5_countries = sorted_data.head(5)

#external style sheets to get custom fonts. I am a typeface nerd!
external_stylesheets = [
    {
        "href": (
            "https://fonts.googleapis.com/css2?"
            "family=IBM+Plex+Sans:wght@400;500&display=swap"
        ),
        "rel": "stylesheet",
    },
]

app = dash.Dash(external_stylesheets=external_stylesheets)

server = app.server

app.layout = html.Div(children=[
    html.Div(children=[
       html.H1(
            children='COVID-19 Dashboard', 
            className='header_title'
        ),
    ], className='header'),
    
   html.Div(
    children=[
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Region", className="menu-title"),
                        dcc.Dropdown(
                            id='scope-dropdown',
                            options=[
                                {'label': 'World', 'value': 'World'},
                                {'label': 'Asia', 'value': 'Asia'},
                                {'label': 'Africa', 'value': 'Africa'},
                                {'label': 'Europe', 'value': 'Europe'},
                                {'label': 'North America', 'value': 'North America'},
                                {'label': 'South America', 'value': 'South America'}
                            ],
                            value="World",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(children="Input Data", className="menu-title"),
                        dcc.RadioItems(
                            id='data-input',
                            options=[
                                {'label': 'Confirmed Cases', 'value': 'ConfirmedCases'},
                                {'label': 'Confirmed Deaths', 'value': 'ConfirmedDeaths'},
                                {'label': 'Stringency Index', 'value': 'StringencyIndex'}
                            ],
                            value='ConfirmedCases',
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Select Policy", className="menu-title"
                        ),
                        dcc.RadioItems(
                            id='policy-input',
                            options=[
                                {'label': 'Not Selected', 'value': 'NotSelected'},
                                {'label': 'School Closing', 'value': 'SchoolClosing'},
                                {'label': 'Staying at Home', 'value': 'StayAtHome'}
                            ],
                            value='NotSelected',
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="world-map",
                        figure=fig
                    ),
                    className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="line-graph",
                        className="card",
                    ),
                    className="card-container",
                ),
            ],
            className="wrapper",
        ),
    ],
    className="container",
)

])

# ----------Callbacks----------------

@app.callback(
    Output('world-map', 'figure'),
    [
        Input('scope-dropdown', 'value'), 
        Input('data-input', 'value'),
        Input('policy-input', 'value'),
        # Input('date-range', 'start_date'),
        # Input('date-range', 'end_date'),
    ]
)

def update_map(scope, data_input, policy_input,):
    filtered_df = merged_df.copy()

    # Filter by scope
    if scope != 'World':
        filtered_df = filtered_df[filtered_df['Continent_Name'] == scope]

    # Sort the DataFrame by 'Date' in ascending order
    filtered_df = filtered_df.sort_values('Date')

    if policy_input != 'NotSelected':
        fig = px.choropleth(
            filtered_df,
            locations='CountryCode',
            locationmode='ISO-3',
            color=policy_input,
            hover_name='CountryName',
            scope=scope.lower(),
            animation_frame=filtered_df['Date'].dt.strftime('%Y-%m-%d'),
            color_continuous_scale=px.colors.sequential.Plasma
        )
        title = '{} Policy by Country ({})'.format(policy_input, scope)
    else:
        fig = px.scatter_geo(
            filtered_df,
            locations='CountryCode',
            locationmode='ISO-3',
            color='Continent_Name',
            hover_name='CountryName',
            size=data_input,
            animation_frame=filtered_df['Date'].dt.strftime('%Y-%m-%d'),
            scope=scope.lower(),
            projection='equirectangular',
            opacity=0.4,
            # size_max=30
        )
        title = 'COVID-19 {} by Country ({})'.format(data_input, scope)
       
    axis_title_font = {'family': 'IBM Plex Sans'} 
    fig.update_layout(
        title={
            'text': title,
            'x': 0.5,
            'xanchor': 'center',
            'font': {'family': 'IBM Plex Sans'},
            'font_color': '#000'
        },
        legend=dict(
            title='CountryName',
            title_font=axis_title_font,
            font_color='#000'
        ),
        geo=dict(
            showocean=True,
            oceancolor="#caf0f8",
            showcountries=True,
            countrycolor="RebeccaPurple",
            showland=True,
            landcolor="#fff",
        ),
    )

    return fig


@app.callback(
    Output('line-graph', 'figure'),
    [
        Input('data-input', 'value'),
        Input('policy-input', 'value'),
    ]
)
def update_line_graph(data_input, policy_input):
    filtered_df = merged_df.copy()

    line_graph = go.Figure()

    for country in top_5_countries['CountryName']:
        country_data = filtered_df[filtered_df['CountryName'] == country]

        if policy_input == 'SchoolClosing':
            y_data = country_data['SchoolClosing']
            yaxis_label = 'School Closing'
        elif policy_input == 'StayAtHome':
            y_data = country_data['StayAtHome']
            yaxis_label = 'Stay at Home Requirements'
        else:
            y_data = country_data[data_input]
            yaxis_label = data_input

        line_graph.add_trace(go.Scatter(
            x=country_data['Date'],
            y=y_data,
            name=country
        ))
        
    axis_title_font = {'family': 'IBM Plex Sans'}
    line_graph.update_layout(
        title={
            'text': '{} in the Top 5 Countries'.format(yaxis_label),
            'x': 0.5,
            'xanchor': 'center',
            'font': {'family': 'IBM Plex Sans'},
            'font_color': '#000'
        },
        xaxis=dict(
            title='Date',
            title_font=axis_title_font,
            color='#000'
        ),
        yaxis=dict(
            title=yaxis_label,
            title_font=axis_title_font,
            color='#000'
        ),
        legend=dict(
            title='CountryName',
            title_font=axis_title_font,
            font_color='#000'
        ),
    )

    return line_graph


if __name__ == "__main__":
    app.run_server(port=8888, debug=True)

### A person's Journey from London - Dover - Calais - Paris - Istanbul

In [99]:
import plotly.graph_objects as go

token = 'pk.eyJ1IjoibWFyc21hdGhldyIsImEiOiJjbGpnNDgwdncwODVyM2dwOTlkYWJwczMyIn0.jGoExgl_LufX90gXa75ceQ'

fig = go.Figure(go.Scattermapbox(
    mode="markers+text+lines",
    lon=[-0.127758, 1.31180, 1.858679, 2.551468871907745, 28.730010839516794],
    lat=[51.507351, 51.12800, 50.952290, 48.8566, 41.0082],
    marker={'size': 18, 'symbol': ["bus", "ferry", "car", "car", "airport"]},
    text=["London", "Dover", "Calais", "Paris", "Istanbul"],
    textposition="bottom right"
))

fig.update_layout(
    title="A person's Journey from London - Dover - Calais - Paris - Istanbul",
    mapbox={
        'accesstoken': token,
        'center': {'lon': 6.7176, 'lat': 49.9719},
        'style': "outdoors",
        'zoom': 5
    },
    hovermode="closest",
)

fig.write_html('./user_journey.html', auto_open=True)


<!-- Add a line graph in a second figure to the web app to compare the number of confirmed cases of the five countries over the given time (i.e. between 1st March and 20th May).
The line graph of the same five countries can be updated based on user’s choice of
• Policy (School closing or Stay at home requirements).
Note1: The y-axis and the title of the line graph should change according to the chosen option e.g. the title is “confirmedDeaths in the top five countries” if the selected option is confirmed Deaths.


• Data input (confirmed cases, confirmed deaths, or stringency index).
map the values in the columns school closing as {0 : 'no measures, 1: 'recommended closing', 2: 'require localised closing', 3: 'require all closing'} -->